In [60]:
import pandas as pd

In [61]:
pod_df = pd.read_csv("../data/pod_data_extract.csv",)

In [62]:
pod_df

,WAYBILL,WAYDATE,DUEDATE,ACCNUM,SERVICE,ORIGPERS,DESTPERS,ORIGHUB,ORIGTOWN,DESTHUB,DESTTOWN,DELIVERYAGENT,PODIMGPRESENT
0,BSC3189560,2025-01-16,NaN,B35,LOC,BSC Stationery Sales,UITKYK SKRYFBEHOEFTES BK,JNB,"CLEVELAND, Johannesburg",PRY,"SILVERTON, Pretoria",DBN ACD - FleetSmart,N
1,I097433,2025-02-06,2025-02-10,G49,RDF,GRAKIM TRADING (PTY) LTD,BEX NEWTON PARK,PRY,"PRETORIA WEST, Pretoria",PLZ,"PORT ELIZABETH, Eastern Cape",JHB ACD - J&T Transport,N
2,BSC3193591,2025-02-11,2025-02-11,B35,KG,BSC Stationery Sales,CIRCULAR OFFICE SUPPLIES & PRINTERS,JNB,"CLEVELAND, Johannesburg",PLZ,"PORT ELIZABETH, Eastern Cape",JHB ACD - J&T Transport,N
3,RES285561,2025-02-17,2025-02-17,U10E,ORA,REBEL ELITE FITNESS,AZAAD SALVATORE - GYM GEAR,JNB,GERMISTON,PLZ,"PORT ELIZABETH, Eastern Cape",JHB ACD - J&T Transport,N
4,WAD-4000178661,2025-02-18,2025-02-20,N30,RDF,NEXUS FULFILMENT,SAB EMLANJENI,JNB,WADEVILLE,DUR,"KWADUKUZA, Stanger",DBN ACD - FleetSmart,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,KYC230425004,2025-04-29,2025-04-29,T70,RDF,TRAGAR LOGISTICS CC,CBS CAPE TOWN,JNB,"JET PARK, Boksburg",CPT,"PINELANDS, Cape Town",CPT OCD - KP40KD GP 8T,N
566,CON060824135,2025-04-29,2025-05-01,T70,PAL,TRAGAR LOGISTICS CC,CONTINENTAL CHINA,JNB,"JET PARK, Boksburg",CPT,"BLACKHEATH, Kuils River",CPT OCD - KP53ZL GP TT,N
567,RMA9915RRS,2025-04-29,2025-05-01,N36CO,RDF,Newedge Technology - RMA Collections,H&H MTHATHA MALL,JNB,"SEBENZA, Edenvale",ELS,MTHATHA,JHB ACD - J&T Transport,N
568,RMA9880RRS,2025-04-29,2025-05-01,N36CO,RDF,Newedge Technology - RMA Collections,OK FURNITURE KLERKSDORP,JNB,"SEBENZA, Edenvale",JNB,KLERKSDORP,JHB ACD - Jordaans's Transport,N


In [63]:
pod_df["WAYDATE"] = pd.to_datetime(pod_df["WAYDATE"])

# Extract date components
pod_df["MONTH_YEAR"] = pod_df["WAYDATE"].dt.to_period("M")
pod_df["DAY"] = pod_df["WAYDATE"].dt.day

pod_df["DELIVERYAGENT"] = pod_df["DELIVERYAGENT"].fillna("No Delivery Agent")

In [64]:
# Get unique month-year combinations
unique_months = pod_df["MONTH_YEAR"].unique()

# Create a dictionary to store dataframes for each month
monthly_dataframes = {}

# Populate the dictionary with dataframes filtered by month
for month in unique_months:
    monthly_dataframes[month] = pod_df[pod_df["MONTH_YEAR"] == month].copy()

# Print the number of dataframes created
print(f"Created {len(monthly_dataframes)} dataframes for the following months:")
for month, df in monthly_dataframes.items():
    print(f"- {month}: {len(df)} records")

Created 4 dataframes for the following months:
- 2025-01: 1 records
- 2025-02: 8 records
- 2025-03: 19 records
- 2025-04: 542 records


In [65]:
import pandas as pd
import calendar


def create_cumulative_pivot_table(
    df: pd.DataFrame,
    index_column: str,
    date_column: str,
) -> pd.DataFrame:
    """
    Create a cumulative pivot table showing daily counts by category.

    Args:
        df: Pre-cleaned DataFrame
        index_column: Column to use for pivot table rows (e.g., 'DELIVERYAGENT')
        date_column: Column containing dates in datetime format
        month_year: Optional period to filter by (e.g., pd.Period('2025-01', 'M'))
                   If None, uses the month from the first date in the dataframe

    Returns:
        DataFrame with cumulative counts pivoted by day of month
    """
    # Convert date column to datetime if it's not already
    if not pd.api.types.is_datetime64_any_dtype(df[date_column]):
        df = df.copy()
        df[date_column] = pd.to_datetime(df[date_column])

    # Extract day from date
    df_with_day = df.copy()
    df_with_day["DAY"] = df_with_day[date_column].dt.day


    first_date = df[date_column].min()
    month_year = pd.Period(f"{first_date.year}-{first_date.month}", freq="M")

    # Get unique categories and days
    categories = df_with_day[index_column].unique()
    _, days_in_month = calendar.monthrange(month_year.year, month_year.month)
    all_days = range(1, days_in_month + 1)

    # Create template with all combinations
    template = pd.MultiIndex.from_product(
        [categories, all_days], names=[index_column, "DAY"]
    )
    template_df = pd.DataFrame(index=template).reset_index()

    # Count by category and day
    count_df = (
        df_with_day.groupby([index_column, "DAY"]).size().reset_index(name="COUNT")
    )

    # Merge to include zeros
    merged_df = pd.merge(template_df, count_df, on=[index_column, "DAY"], how="left")
    merged_df["COUNT"] = merged_df["COUNT"].fillna(0)

    # Calculate cumulative sum
    merged_df = merged_df.sort_values([index_column, "DAY"])
    merged_df["CUMULATIVE"] = merged_df.groupby(index_column)["COUNT"].cumsum()

    # Create pivot table
    pivot_df = merged_df.pivot(index=index_column, columns="DAY", values="CUMULATIVE")

    # Forward fill NaNs (for days with no activity)
    pivot_df = pivot_df.ffill(axis=1)

    # Fill remaining NaNs with 0
    pivot_df = pivot_df.fillna(0)

    return pivot_df

In [66]:
pivot_dfs = {}
for df in monthly_dataframes.values():
    pivot_dfs[df["MONTH_YEAR"].iloc[0]] = create_cumulative_pivot_table(
        df=df, index_column="DELIVERYAGENT", date_column="WAYDATE"
    )

In [67]:
# Save all pivot tables to a single Excel file with multiple sheets
excel_file = "../monthly_delivery_agent_cumulative_counts.xlsx"

# Use ExcelWriter to save multiple dataframes to different sheets
with pd.ExcelWriter(excel_file, engine="openpyxl") as writer:
    for month, pivot_df in pivot_dfs.items():
        # Convert period to string for sheet name
        sheet_name = f"{month.year}_{month.month:02d}"

        # Write the dataframe to a sheet
        pivot_df.to_excel(writer, sheet_name=sheet_name)

print(f"All pivot tables saved to {excel_file}")

All pivot tables saved to ../monthly_delivery_agent_cumulative_counts.xlsx
